# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.



---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

---
**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Import Packages

In [1]:
#importing some useful packages
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as mpatch
from scipy.stats import norm
import numpy as np
import math
import cv2

from IPython.display import display, HTML
from moviepy.editor import VideoFileClip
%matplotlib inline

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [2]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, [vertices], ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def maskRegion(img, xshift=0, yshift=20):
    xsize = img.shape[1]
    ysize = img.shape[0]
    vertices = np.array([ [0,ysize],[xsize,ysize],
                         [xsize/2+xshift,ysize/2+yshift],[xsize/2-xshift,ysize/2+yshift] ], dtype=np.int32)
    return vertices

def draw_lines(img, lines, color=[255, 0, 0], thickness=2, ydraw_shift=60):
    """
    Average/extrapolate the line segments you detect to map out the full
    extent of the lane. Separating line segments by their slope ((y2-y1)/(x2-x1)) 
    to decide which segments are part of the left line vs. the right line.  
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    """
    x_left = []
    y_left = []
    x_right = []
    y_right = []
    for line in lines:
        for x1,y1,x2,y2 in line:
#             cv2.line(img,(x1,y1),(x2,y2), color, thickness)
            slope = ((y2-y1)/(x2-x1))
            if -0.3 < slope < 0.3: # filter out extra lines
                continue
            elif slope < 0:
                x_left.extend([x1,x2])
                y_left.extend([y1,y2])
            else:
                x_right.extend([x1,x2])
                y_right.extend([y1,y2])
    left_line = average_lines(img, x_left, y_left, ydraw_shift)
    right_line = average_lines(img, x_right, y_right, ydraw_shift)
    cv2.line(img, left_line[0], left_line[1], color, thickness)
    cv2.line(img, right_line[0], right_line[1], color, thickness)
            
def average_lines(img, x, y, ydraw_shift):
    """From lists of points x,y return the start and end point from averaged line."""
    ymax = img.shape[0]
    ymin = int(img.shape[0]/2 + ydraw_shift) 
    
    slope, intersect = np.polyfit(x,y,1)
    xmax = int((ymax-intersect)/slope)
    xmin = int((ymin-intersect)/slope)
    return [(xmin,ymin),(xmax,ymax)]

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, color=[255, 0, 0], thickness=2):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, color, thickness)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(initial_img, img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


def drawMaskRegion(img, pts, close=True, color=(255,0,0), thickness=3):
    img = np.dstack((img,img,img))
    cv2.polylines(img, [pts], close, color, thickness)
    return img


def displayImgs(imgnames, imgdir):
    """Display a list of images as HTML embeded form"""
    html = "<div>"
    for img in imgnames:
        source = imgdir + img
        html += """<img src="{0}" width="300" height="120" style="float:left; margin:2px"/>""".format(source)
    html += "</div>"
    display(HTML(html))
    
def saveImgs(imgnames, savedir, images, isGray=0):
    """Save output images to the defined directory."""
    if not os.path.exists(savedir):
        os.makedirs(savedir)
    for i, img in enumerate(imgnames):
        imgp = savedir + img
        if isGray:
            plt.imsave(imgp, images[i], cmap='Greys_r')
        else:
            plt.imsave(imgp, images[i])
            
def groupAction(action, targets):
    """Apply the same function/action on a list of targets."""
    return list(map(action, targets))

def renameImgs(imgnames, addstr):
    """Modify a list of image names."""
    return list(map(lambda x: addstr+x, imgnames))

## Build a Lane Finding Pipeline

Build the pipeline and run solution on all test_images. Make copies into the `test_images_output` directory.

### Load test images


In [4]:
# load images from given directory "test_images/"
imgdir = "test_images/"
imgnames = os.listdir(imgdir)

loadAct = lambda img: mpimg.imread(imgdir + img) # cv2.imread() read to BGR instead of RGB
testImgs = groupAction(loadAct, imgnames)

displayImgs(imgnames, imgdir)

### Converting images to gray scale

In [5]:
grayImgs = groupAction(grayscale, testImgs)

savedir = "test_images_output/"
grayImgnames = renameImgs(imgnames, "gray_")
saveImgs(grayImgnames, savedir, grayImgs, isGray=1)

displayImgs(grayImgnames, savedir)

### Gaussian smoothing on gray scale images

In [6]:
kernal_size = 7
smoothAct = lambda img: gaussian_blur(img, kernal_size)
smoothImgs = groupAction(smoothAct, grayImgs)

smoothImgnames = renameImgs(imgnames, "smooth_")
saveImgs(smoothImgnames, savedir, smoothImgs, isGray=1)

displayImgs(smoothImgnames, savedir)

### Canny edge detection

In [7]:
low_threshold = 80
high_threshold = 180 
cannyAct = lambda img: cv2.Canny(img, low_threshold, high_threshold)
cannyImgs = groupAction(cannyAct, smoothImgs)

cannyImgnames = renameImgs(imgnames, "canny_")
saveImgs(cannyImgnames, savedir, cannyImgs)

displayImgs(cannyImgnames, savedir)

### Masking region of interest

In [8]:
xshift = 30
yshift = 60
maskAct = lambda img: region_of_interest(img, maskRegion(img, xshift, yshift))
maskedImgs = groupAction(maskAct, cannyImgs)

# add the mask boundary for display to easier tune the parameters
mask2showAct = lambda img: drawMaskRegion(img, maskRegion(img, xshift, yshift))
maskedImgs2show = groupAction(mask2showAct, maskedImgs)

maskedImgnames = renameImgs(imgnames, "mask_")
saveImgs(maskedImgnames, savedir, maskedImgs2show)

displayImgs(maskedImgnames, savedir)

### Hough transform and line extrapolate

In [9]:
rho          = 1 # pixels of Hough grid
theta        = np.pi/180 # angle of Hough grid
threshold    = 20 # min votes for a line within a grid
min_line_len = 30 # min number of pixels for a line
max_line_gap = 15 # max gap in pixels for joinable line segments

color = (255,0,0)
thickness = 15

houghAct = lambda img: hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, color, thickness)
houghImgs = groupAction(houghAct, maskedImgs)

houghImgnames = renameImgs(imgnames, "hough_")
saveImgs(houghImgnames, savedir, houghImgs)

displayImgs(houghImgnames, savedir)

### Display weighted image

In [10]:
α = 1.0
β = 0.8
dispAct = lambda img: weighted_img(img[0], img[1], α, β)
feed = zip(houghImgs, testImgs)
dispImgs = groupAction(dispAct, feed)

dispImgnames = renameImgs(imgnames, "disp_")
saveImgs(dispImgnames, savedir, dispImgs)

displayImgs(dispImgnames, savedir)

### Darw lane lines pipeline function

In [52]:
# The pipeline that will draw lane lines on the test_images
def draw_laneLines_pipe(imgpath, **kwargs):
    img = mpimg.imread(imgpath)
    grayimg = grayscale(img)
    blurimg = gaussian_blur(grayimg, kwargs['kernal'])
    edgeimg = cv2.Canny(img, kwargs['low_threshold'], kwargs['high_threshold'])
    region = maskRegion(edgeimg, kwargs['xshift'], kwargs['yshift'])
    maskimg = region_of_interest(edgeimg, region)
    lineimg = hough_lines(maskimg, kwargs['rho'],kwargs['theta'],kwargs['threshold'],
                          kwargs['min_line_len'],kwargs['max_line_gap'],kwargs['line_color'],kwargs['line_width'])
    dispimg =  weighted_img(img, lineimg, kwargs['baseimg_alpha'], kwargs['lineimg_alpha'])
    return dispimg

# params = {
# 'kernal': 7,
# 'low_threshold': 60,
# 'high_threshold': 150,
# 'xshift': 10,
# 'yshift': 30,
# 'rho': 1, 
# 'theta': np.pi/180,
# 'threshold': 10,
# 'min_line_len': 30,
# 'max_line_gap': 15,
# 'line_color': (255,0,0),
# 'line_width': 15,
# 'baseimg_alpha': 1.0,
# 'lineimg_alpha': 0.6,
# }

# imgdir = "./test_images/"
# for ip in list(map(lambda p: imgdir+p, os.listdir(imgdir))):
#     out = draw_laneLines_pipe(ip, **params)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

### Test on `solidWhiteRight.mp4`

Let's try the one with the solid white lane on the right first ...

In [11]:
def process_image(image):
    # Return the final output as colored image where lines are drawn on lanes.
    kwargs = {
    'kernal': 7,
    'low_threshold': 60,
    'high_threshold': 150,
    'xshift': 10,
    'yshift': 30,
    'rho': 1, 
    'theta': np.pi/180,
    'threshold': 10,
    'min_line_len': 30,
    'max_line_gap': 15,
    'line_color': (255,0,0),
    'line_width': 15,
    'baseimg_alpha': 1.0,
    'lineimg_alpha': 0.6,
    }
    grayimg = grayscale(image)
    blurimg = gaussian_blur(grayimg, kwargs['kernal'])
    edgeimg = cv2.Canny(image, kwargs['low_threshold'], kwargs['high_threshold'])
    region = maskRegion(edgeimg, kwargs['xshift'], kwargs['yshift'])
    maskimg = region_of_interest(edgeimg, region)
    lineimg = hough_lines(maskimg, kwargs['rho'],kwargs['theta'],kwargs['threshold'],
                          kwargs['min_line_len'],kwargs['max_line_gap'],kwargs['line_color'],kwargs['line_width'])
    result =  weighted_img(image, lineimg, kwargs['baseimg_alpha'], kwargs['lineimg_alpha'])
    return result


In [13]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
# clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5) #pipeline on a shorter subclip, .subclip(start_second,end_second)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
# white_clip = clip1.fx(transform_clip, **params) # Tried to passing extra parameters in the function
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:04<00:00, 49.39it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 2.91 s, sys: 600 ms, total: 3.51 s
Wall time: 4.98 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [14]:
HTML("""
<video width="960" height="540" controls>  <source src="{0}">  </video>
""".format(white_output))

###  Test on `solidYellowLeft.mp4`

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [24]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:14<00:00, 47.01it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 9.4 s, sys: 1.82 s, total: 11.2 s
Wall time: 15 s


In [26]:
HTML("""
<video width="960" height="540" controls>  <source src="{0}"> </video>
""".format(yellow_output))

### Test summary
The pipeline works good for the first video `solidWhiteRight.mp4`, however for `solidYellowLeft.mp4`, there are twice the right lane line is off the right position. The possible causes of this are:

1. incontinuous of the lane line
2. disturbing patterns on the ground
3. less clear edges for the lane lines

Similar problems occurred to the `challenge.mp4` video below. The are a few possible ways to improve the pipeline:

1. improve the contrast of the picture for edge detection
2. a better region of interest selection for different video scales
3. new algorithms for extrapolate lane lines

## Optional Challenge

Try your lane finding pipeline on the video below. With modified pipeline from above.

### Modifying some functions

Some exploration of making it better, BUT... all did not work...

In [9]:
# change 1: limit a smaller range of region of interest
def maskRegion(img, xshift=(0,0), yshift=(0,1/8)):
    xsize = img.shape[1]
    ysize = img.shape[0]
    xshift1 = int(xshift[0]*xsize)
    xshift2 = int(xshift[1]*xsize)
    yshift1 = int(yshift[0]*ysize)
    yshift2 = int(yshift[1]*ysize)
    vertices = np.array([ [0+xshift1,       ysize-yshift1],
                          [xsize-xshift1,   ysize-yshift1],
                          [xsize/2+xshift2, ysize/2+yshift2],
                          [xsize/2-xshift2, ysize/2+yshift2] ], dtype=np.int32)
    return vertices

def draw_lines(img, lines, line_hist, color=[255, 0, 0], width=2, ydraw_shift=50):
    """
    Average/extrapolate the line segments you detect to map out the full
    extent of the lane. Separating line segments by their slope ((y2-y1)/(x2-x1)) 
    to decide which segments are part of the left line vs. the right line.  
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    """
    x_left = []
    y_left = []
    x_right = []
    y_right = []
    for line in lines:
        for x1,y1,x2,y2 in line:
#             cv2.line(img,(x1,y1),(x2,y2), color, width)
            slope = ((y2-y1)/(x2-x1))
            if -0.3 < slope < 0.3: # filter out extra lines
                continue
            elif slope < 0:
                x_left.extend([x1,x2])
                y_left.extend([y1,y2])
            else:
                x_right.extend([x1,x2])
                y_right.extend([y1,y2])
                
    left_line   = average_lines(img, x_left, y_left, ydraw_shift)
    right_line  = average_lines(img, x_right, y_right, ydraw_shift)
    
#     if left_line[0][0]==0:
#         left_line = line_hists['left']
#     elif right_line[0][0]==0:
#         right_line = line_hists['right']
    
    cv2.line(img, left_line[0], left_line[1], color, width)
    cv2.line(img, right_line[0], right_line[1], color, width)
    
#     line_hists['left'] = left_line
#     line_hists['right'] = right_line
            

def average_lines(img, x, y, ydraw_shift):
# def average_lines(img, x, y, line_hist, ydraw_shift):
    """From lists of points x,y return the start and end point from averaged line."""

#     if len(line_hist) < 5:
#         line_hist.append( (slope, intersect) )
#     else:
#         last_slope = line_hist[-1][0]
#         last_intersect = line_hist[-1][1]
#         old_slopes, old_intersects = zip(*line_hist)
#         slope_diff = avg_diff(old_slopes)
#         intersect_diff = avg_diff(old_intersects)
#         last_slope_diff = diff_rate(line_hist[-2][0], last_slope)
#         last_intersect_diff = diff_rate(line_hist[-2][1], last_intersect)
        
#         if abs(diff_rate(last_slope, slope)) > 2*slope_diff:
#             slope = apply_avg_diff(last_slope_diff, last_slope)
#             intersect = apply_avg_diff(last_intersect_diff, last_intersect)
        
#         line_hist.pop(0)
#         line_hist.append( (slope,intersect) )

    ymax = img.shape[0]
    ymin = int(img.shape[0]/2 + ydraw_shift) 
#     try:
    slope, intersect = np.polyfit(x,y,1)
    xmax = int((ymax-intersect)/slope)
    xmin = int((ymin-intersect)/slope)
#     except:
#         xmin = 0
#         xmax = 0
    return [(xmin,ymin),(xmax,ymax)]

# def diff_rate(before, after):
#     """calculating the ratio of change."""
#     return (after - before)/before

# def avg_diff(listnum):
#     """calculating the average difference of a list."""
#     diffs = list(map(lambda x,y: diff_rate(x,y), listnum[:-1], listnum[1:]))
#     return np.average(diffs)

# def apply_avg_diff(diff, num):
#     """calculate the next number based on changing rate"""
#     return num*(1 + diff)


def hough_lines(img, **kwargs):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, kwargs['rho'], kwargs['theta'],kwargs['threshold'], np.array([]), 
                            minLineLength=kwargs['min_line_len'], maxLineGap=kwargs['max_line_gap'])
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, kwargs['line_color'], kwargs['line_width'], kwargs['ydraw_shift'])
    return line_img

def new_process_image(image):
    # Return the final output as colored image where lines are drawn on lanes.
    kwargs = {
    'kernal': 11,
    'low_threshold': 80,
    'high_threshold': 150,
    'xshift': (1/30, 1/80),
    'yshift': (0, 1/10),
    'hough':{
        'rho': 1, 
        'theta': np.pi/180,
        'threshold': 15,
        'min_line_len': 15,
        'max_line_gap': 15,
        'line_color': (255,0,0),
        'line_width': 12,
        'ydraw_shift': 90,
        },
    'baseimg_alpha': 1.0,
    'lineimg_alpha': 0.6,
    }
    grayimg = grayscale(image)
    blurimg = gaussian_blur(grayimg, kwargs['kernal'])
    edgeimg = cv2.Canny(image, kwargs['low_threshold'], kwargs['high_threshold'])
    region = maskRegion(edgeimg, kwargs['xshift'], kwargs['yshift'])
    maskimg = region_of_interest(edgeimg, region)
    lineimg = hough_lines(maskimg, **kwargs['hough'])
    result =  weighted_img(image, lineimg, kwargs['baseimg_alpha'], kwargs['lineimg_alpha'])
    return result


# def transform_clip(clip, arg):
#     def new_process(image):
#         return new_process_image(image, arg)
#     return clip.fl_image(new_process)

### Test on `challenge.mp4`

In [16]:
# line_hists = {'left':[],'right':[]} # record some old slopes to smooth frames
# global  line_hists

challenge_output = 'test_videos_output/challenge.mp4'
# clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,3)
clip3 = VideoFileClip('test_videos/challenge.mp4')

# challenge_clip = clip3.fx(transform_clip, line_hists)
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge2.mp4
[MoviePy] Writing video test_videos_output/challenge2.mp4


 29%|██▉       | 74/251 [00:02<00:07, 24.89it/s]WARNING:py.warnings:/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:66: RuntimeWarning: divide by zero encountered in int_scalars

100%|██████████| 251/251 [00:10<00:00, 26.59it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge2.mp4 

CPU times: user 7.19 s, sys: 950 ms, total: 8.14 s
Wall time: 11.3 s


In [17]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))